In [1]:


import numpy as np

from spn.algorithms.StructureLearning import get_next_operation, learn_structure
from spn.algorithms.CnetStructureLearning import get_next_operation_cnet, learn_structure_cnet
from spn.algorithms.Validity import is_valid
from spn.algorithms.Statistics import get_structure_stats_dict

from spn.structure.Base import Sum, assign_ids

from spn.structure.leaves.histogram.Histograms import create_histogram_leaf
from spn.structure.leaves.parametric.Parametric import create_parametric_leaf
from spn.structure.leaves.piecewise.PiecewiseLinear import create_piecewise_leaf
from spn.structure.leaves.cltree.CLTree import create_cltree_leaf
from spn.algorithms.splitting.Conditioning import (
	get_split_rows_naive_mle_conditioning,
	get_split_rows_random_conditioning,
)

from spn.algorithms.splitting.Clustering import get_split_rows_XMeans
from spn.algorithms.splitting.RDC import get_split_cols_single_RDC_py, get_split_cols_distributed_RDC_py

import logging

logger = logging.getLogger(__name__)


import warnings

warnings.filterwarnings('ignore')



import pandas as pd
from spn.structure.Base import Context
from spn.structure.StatisticalTypes import MetaType
import matplotlib.pyplot as plt
from os import path as pth
import sys, os

cols="rdc"
rows="kmeans"
min_instances_slice=200
threshold=0.3
ohe=False
leaves = create_histogram_leaf
rand_gen=None
cpus=-1


#datasets = ["nltcs","msnbc", "kdd", "plants", "baudio", "jester", "bnetflix"]
datasets = ["nltcs"]
path = "test1"

for dataset in datasets:
    
    print(f"\n\n\n{dataset}\n\n\n")
    plot_path = f"{path}/{dataset}"
    if not pth.exists(plot_path):
        try:
            os.makedirs(plot_path)
        except OSError:
            print ("Creation of the directory %s failed" % plot_path)
            sys.exit()
            
    df = pd.read_csv(f"spn/data/binary/{dataset}.ts.data", sep=',')
    data = df.values
    print(data.shape)
    max_iter = data.shape[1]
    samples, var = data.shape
    ds_context = Context(meta_types=[MetaType.DISCRETE]*var)
    ds_context.add_domains(data)

    df2 = pd.read_csv(f"spn/data/binary/{dataset}.test.data", sep=',')
    test = df2.values[:1000]
    print(test.shape)

    ll = list()
    nodes = list()
    k1 = 2 #[i for i in range(1,5)]
    past3 = list()
    
    n = int(max_iter**0.5)  #[i for i in range(int(max_iter**0.5),max_iter+1,2)]
    step = (max_iter - (max_iter**0.5))/20

    i,j,k = 0,0,0
    while True:
        split_cols = get_split_cols_distributed_RDC_py(rand_gen=rand_gen, ohe=ohe, n_jobs=cpus, n=round(n))
        split_rows = get_split_rows_XMeans(limit=k1, returnk=False)
        nextop = get_next_operation(min_instances_slice)

        spn = learn_structure(data, ds_context, split_rows, split_cols, leaves, nextop)

        nodes.append(get_structure_stats_dict(spn)["nodes"])
        from spn.io.Graphics import plot_spn

        plot_spn(spn, f'{path}/{dataset}/spn{k}.png')

        from spn.algorithms.Inference import log_likelihood
        total_ll = 0
        for instance in test:
            import numpy as np
            test_data = np.array(instance).reshape(-1, var)
            total_ll += log_likelihood(spn, test_data)[0][0]
        ll.append(total_ll/len(test))
        
        if len(ll)>3:
            past3 = ll[-3:]
            if round(np.std(past3), 2) <= 0.01:
                break

        '''
        if n==max_iter:
            break
        '''
        print("\n\n\n\n\n")
        print(k1,round(n))
        print(nodes[k])
        print(ll[k])
        print(ll)
        print(nodes)
        print("\n\n\n\n\n")
        
        k+=1
        
        plt.close()
        # plot line 
        plt.plot(ll, marker="o") 
        plt.title(f"{dataset} Log Likelihood")
        plt.savefig(f"{path}/{dataset}/ll.png", dpi=100)
        plt.close()
        plt.plot(nodes, marker="o") 
        plt.title(f"{dataset} Nodes")
        plt.savefig(f"{path}/{dataset}/nodes.png", dpi=100)
        plt.close()
        
        
        n = min(n+step, max_iter)
        k1 += 1

    print("Log Likelihood",ll)
    print("Nodes",nodes)

    plt.close()
    # plot line 
    plt.plot(ll, marker="o") 
    #plt.show()
    plt.title(f"{dataset} Log Likelihood")
    plt.savefig(f"{path}/{dataset}/ll.png", dpi=100)
    plt.close()
    plt.plot(nodes, marker="o") 
    #plt.show()
    plt.title(f"{dataset} Nodes")
    plt.savefig(f"{path}/{dataset}/nodes.png", dpi=100)
    plt.close()





nltcs



(16180, 16)
(1000, 16)






2 4
54
-6.804741256769169
[-6.804741256769169]
[54]












3 5
67
-6.457895333193961
[-6.804741256769169, -6.457895333193961]
[54, 67]












4 5
93
-6.245021133171585
[-6.804741256769169, -6.457895333193961, -6.245021133171585]
[54, 67, 93]












5 6
111
-6.1945954933962435
[-6.804741256769169, -6.457895333193961, -6.245021133171585, -6.1945954933962435]
[54, 67, 93, 111]












6 6
113
-6.123549773647917
[-6.804741256769169, -6.457895333193961, -6.245021133171585, -6.1945954933962435, -6.123549773647917]
[54, 67, 93, 111, 113]












7 7
130
-6.078926165613225
[-6.804741256769169, -6.457895333193961, -6.245021133171585, -6.1945954933962435, -6.123549773647917, -6.078926165613225]
[54, 67, 93, 111, 113, 130]












8 8
147
-6.073195575717702
[-6.804741256769169, -6.457895333193961, -6.245021133171585, -6.1945954933962435, -6.123549773647917, -6.078926165613225, -6.073195575717702]
[54, 67, 93, 111, 113, 130, 147]
